### Script to run all prompts on the data set ###

In [ ]:
import os
import pandas as pd
import numpy as np
import logging

logger = logging.getLogger(__name__)

# Import this module with autoreload
%load_ext autoreload
%autoreload 2
import llmt
from llmt.llmtools import Prompt
from llmt.llmtools import MentalHealth, OutpatientServices
from llmt.llmtools import process_prompt
from llmt.openai import OpenAI, create_messages
from llmt.performance import Performance

In [ ]:
# Parameters
model = 'gpt-4o'
temperature = 0

# Directories and files
data_dir = os.path.join(os.environ.get('HOME'), 'home_data', 'hcp')
test_file_name = 'hcp-alldata-250413.parquet'
test_file = os.path.join(data_dir, test_file_name)
df_all = pd.read_parquet(test_file)
# Filter the labeled data
df_train = df_all.loc[df_all['dset'] == 'train'].\
                reset_index(drop=True).\
                astype({'mental_health': int,
                        'inpatient': int,
                        'outpatient': int})
display(df_train.head())
print(df_train.shape)

In [ ]:
# Add some additional samples 
test_samples = 5
random_state = 234
df_test = df_all.loc[df_all['mental_health'].isnull()].\
                sample(n=test_samples, replace=False, random_state=random_state).\
                reset_index(drop=True)
# Combine the training and test samples
df = pd.concat([df_train, df_test], axis=0, ignore_index=True).\
                sample(frac=1, random_state=random_state).\
                reset_index(drop=True)
for dset in ['train', 'test']:
    print(f'{dset}: {len(df.loc[df['dset']==dset, 'id'].unique())}')

display(df.head())

In [ ]:
def predict_mh(name, description, variable, version, client=None): 
    model = 'gpt-4o'
    temperature = 0
    response_format = MentalHealth
    prompt_name = f'{variable}_system_{str(prompt_version).zfill(2)}'
    system_prompt = Prompt().load(prompt_name=prompt_name)
    user_prompt = process_prompt(f"""
                    The organization {name} is described as: {description} 
                    Does this organization provide {variable} healthcare services?
                    """)
    messages = create_messages(system_prompt=system_prompt, user_prompt=user_prompt)
    output = OpenAI().send_messages(messages=messages,
                                    model=model,
                                    temperature=temperature,
                                    response_format=response_format,
                                    client=client)
    # Replace the boolean with binary outcome prediction
    key = 'pred_mh'
    output.update({key: 1 if output.get(key) == True else 0})
    # Select the fields that we want 
    output = {key: output.get(key)}
    return output



In [ ]:
company_id_list = sorted(list(df['id'].unique()))
company_id = company_id_list[10]
df_id = df.loc[df['id'] == company_id]
display(df_id)
variable = 'mental_health'
version = 2
name = df_id['name'].values[0]
description = df_id['description'].values[0]
output = predict_mh(name=name, description=description, variable=variable, version=version)
print(output)

In [ ]:
display(output)

In [ ]:
# Run the prompt on all data
company_id_list = sorted(list(df['id'].unique()))
results_df_list = []

for c, company_id in enumerate(company_id_list):
    if (c + 1) % 20 == 0:
        print(f'Sending description {c + 1} / {len(company_id_list)} to the model')
    df_id = df.loc[df['id'] == company_id]

    # Mental health predictions
    df_id = df_id.assign(**mh_output)
    
    # Inpatient predictions
    df_id = df_id.assign(**ip_output)
    
    # Outpatient predictions
    df_id = df_id.assign(**op_output)

    # Add the new data frame to the list
    result_df_list.append(df_id)

results_df = pd.concat(results_df_list, axis=0, ignore_index=True)
# Save the results
results_file_name = f'{variable}_{str(prompt_version).zfill(2)}_results.parquet'
results_file = os.path.join(data_dir, results_file_name)
results_df.to_parquet(results_file)
print(results_file)